<a href="https://colab.research.google.com/github/Precyors/NLP/blob/main/MarianMT_FineTuning_MachineTranslation_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers datasets sentencepiece torch sacrebleu scikit-learn evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
 # Imports
from transformers import MarianTokenizer, MarianMTModel, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
# Load MarianMT model (EN -> FR) ===
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:

# Create domain-specific dataset (medical)
data = {
    'en': [
        "Take two pills daily.",
        "Consult your doctor if symptoms persist.",
        "Apply this cream to the affected area twice a day.",
        "Do not consume alcohol while taking this medication.",
        "Follow up in one week for a check-up.",
        "Drink plenty of water.",
        "Rest for at least 8 hours daily.",
        "Avoid heavy exercise during recovery.",
        "Take this medicine after meals.",
        "Monitor your blood pressure regularly."
    ],
    'fr': [
        "Prenez deux pilules par jour.",
        "Consultez votre médecin si les symptômes persistent.",
        "Appliquez cette crème sur la zone affectée deux fois par jour.",
        "Ne consommez pas d'alcool pendant la prise de ce médicament.",
        "Faites un suivi dans une semaine pour un contrôle.",
        "Buvez beaucoup d'eau.",
        "Reposez-vous au moins 8 heures par jour.",
        "Évitez les exercices intenses pendant la récupération.",
        "Prenez ce médicament après les repas.",
        "Surveillez régulièrement votre tension artérielle."
    ]
}

df = Dataset.from_dict(data)

In [ ]:
# Split into train/test (80/20)
train_df, test_df = train_test_split(data['en'], test_size=0.2, random_state=42)
train_labels, test_labels = train_test_split(data['fr'], test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict({'en': train_df, 'fr': train_labels})
test_dataset = Dataset.from_dict({'en': test_df, 'fr': test_labels})

In [ ]:
 # Tokenize
def preprocess(batch):
    inputs = tokenizer(batch['en'], truncation=True, padding='max_length', max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch['fr'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_test = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=5,
    logging_steps=5,
    save_steps=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    processing_class=tokenizer
)

In [ ]:
# Baseline Translation (Before Fine-Tuning)
def translate(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

print("=== Baseline Translations ===")
baseline_preds = translate(list(test_dataset['en']), model, tokenizer)
for en, fr in zip(test_dataset['en'], baseline_preds):
    print(f"EN: {en}")
    print(f"FR: {fr}")
    print("------")

=== Baseline Translations ===
EN: Take this medicine after meals.
FR: Prenez ce médicament après les repas.
------
EN: Consult your doctor if symptoms persist.
FR: Consultez votre médecin si les symptômes persistent.
------


In [ ]:
#  Fine-tune the model
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss,Validation Loss
5,2.760700,0.112186
10,0.160600,0.076039
15,0.091900,0.050199
20,0.065300,0.042217


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=20, training_loss=0.7696049809455872, metrics={'train_runtime': 21.2451, 'train_samples_per_second': 1.883, 'train_steps_per_second': 0.941, 'total_flos': 1355934597120.0, 'train_loss': 0.7696049809455872, 'epoch': 5.0})

In [ ]:
# Post-Fine-Tuning Translation
print("\n=== Post-Fine-Tuning Translations ===")
fine_tuned_preds = translate(list(test_dataset['en']), model, tokenizer)
for en, fr in zip(test_dataset['en'], fine_tuned_preds):
    print(f"EN: {en}")
    print(f"FR: {fr}")
    print("------")


=== Post-Fine-Tuning Translations ===
EN: Take this medicine after meals.
FR: Prenez ce médicament après les repas.
------
EN: Consult your doctor if symptoms persist.
FR: Consultez votre médecin si les symptômes persistent.
------


In [ ]:
# BLEU Score Evaluation
sacrebleu = evaluate.load('sacrebleu')
baseline_bleu = sacrebleu.compute(predictions=baseline_preds, references=[[r] for r in test_dataset['fr']])
fine_tuned_bleu = sacrebleu.compute(predictions=fine_tuned_preds, references=[[r] for r in test_dataset['fr']])
print(f"Baseline BLEU: {baseline_bleu['score']:.2f}")
print(f"Fine-tuned BLEU: {fine_tuned_bleu['score']:.2f}")

Baseline BLEU: 100.00
Fine-tuned BLEU: 100.00
